In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
###climate_entities
import pandas as pd

climate_entities_df = pd.read_csv('/content/drive/MyDrive/Labeled Dataset/extracted_unique_entities.csv')

climate_entities = set(entity for entity in climate_entities_df['entities'].tolist() if isinstance(entity, str))



data = pd.read_csv('/content/drive/MyDrive/Labeled Dataset/climate_agreement.csv')

# check and extract entities mentioned in the text
def extract_entities(text):
    if pd.isnull(text):  # Check if text is NaN
        return []
    return [entity for entity in climate_entities if entity in text]

# Extract entities for each row's 'body_parent' and 'body_child'
data['entity_parent'] = data['body_parent'].apply(extract_entities)
data['entity_child'] = data['body_child'].apply(extract_entities)

filtered_data = data[(data['entity_parent'].apply(len) > 0) & (data['entity_child'].apply(len) > 0)]

# Combine entities and remove duplicates
filtered_data = filtered_data.copy()  # Make a deep copy
filtered_data['entity'] = filtered_data.apply(lambda row: sorted(set(row['entity_parent'] + row['entity_child'])), axis=1)

exploded_data = filtered_data.explode('entity')

result = exploded_data[['msg_id_parent', 'msg_id_child', 'body_parent', 'body_child', 'label', 'entity', 'exact_time', 'datetime']]

unwanted_entities = [', ', 'n', 'N', 'P', 'F', 's']
result = result[~result['entity'].isin(unwanted_entities) & ~(result['entity'].str.len() == 1)]

result.reset_index(drop=True, inplace=True)


result.to_csv('/content/drive/MyDrive/Labeled Dataset/long_filtered_climate.csv', index=False)




In [ ]:
pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.8 MB/s eta 0:00:00


In [ ]:
#compute entity-based sentiment scores, An entity may only appears in either body parent or body child in one pair, but we
# want to compute the sentiment scores of both texts separately towards one entity. So we stick with the sentiment analysis pipeline from HuggingFace's Transformers. The function will be similar to before but will return a neutral score for texts that don't have the entity.
import pandas as pd
from transformers import pipeline

entities_df = pd.read_csv('/content/drive/MyDrive/Labeled Dataset/extracted_unique_entities.csv')
climate_entities = entities_df['entities'].dropna().astype(str).tolist()


data = pd.read_csv('/content/drive/MyDrive/Labeled Dataset/long_filtered_climate.csv')

# Initialize the sentiment-analysis pipeline from HuggingFace's Transformers
sentiment_pipeline = pipeline("sentiment-analysis")

def compute_overall_sentiment(text):
    # Check for NaN or None values
    if pd.isna(text):
        return None


    sentiment = sentiment_pipeline(text)[0]

    if sentiment['label'] == "POSITIVE":
        return sentiment['score']
    elif sentiment['label'] == "NEGATIVE":
        return -sentiment['score']
    else:
        return 0.0

def compute_entity_sentiment(text, entity):
    if entity in text:
        start_idx = max(text.find(entity) - 30, 0)  # 30 characters before the entity
        end_idx = min(text.find(entity) + len(entity) + 30, len(text))  # 30 characters after the entity
        context = text[start_idx:end_idx]
        sentiment = sentiment_pipeline(context)[0]
        return sentiment['label'], sentiment['score']
    else:
        return "NEUTRAL", 0.0

# Calculate overall sentiment scores for 'body_parent' and 'body_child'
data['sentiment_parent'] = data['body_parent'].apply(compute_overall_sentiment)
data['sentiment_child'] = data['body_child'].apply(compute_overall_sentiment)


for entity in climate_entities:

    data[f'sentiment_parent_{entity}'], data[f'score_parent_{entity}'] = zip(*data['body_parent'].apply(lambda x: compute_entity_sentiment(x, entity)))


    data[f'sentiment_child_{entity}'], data[f'score_child_{entity}'] = zip(*data['body_child'].apply(lambda x: compute_entity_sentiment(x, entity)))


data.to_csv('/content/drive/MyDrive/Labeled Dataset/long_sentiments.csv', index=False)


In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.9 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125924 sha256=6f7f1ce978b189dcd8e1cf3a072f8213d0335b776f2f2718795bd6d829b2a9c8
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
